<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex11notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex11notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo11.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# scikit-learn の判別分析のためのクラス
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# scikit-learn の主成分分析のためのクラス
from sklearn.decomposition import PCA

----
## 手書き数字認識してみよう
----


---
### データの準備


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
X_train = rv['datL'].astype(float)
Y_train = rv['labL']
X_test = rv['datT'].astype(float)
Y_test = rv['labT']
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

K = 10 # クラス数
N_train, D = X_train.shape
N_test, _  = X_test.shape

どんデータなのか知るために，一部を可視化してみましょう．

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
# 学習データの最初の nx * ny 個を画像として表示
nx, ny = 10, 5
display(X_train[:ny*nx], nx, ny)

# それらのクラスラベルを表示
for iy in range(ny):
    print(Y_train[iy*nx:(iy+1)*nx])

---
### 実験その1



#### 問題1

次のセルに，次のことを行うコードを書きなさい
1. 学習データを用いて線形判別分析（パラメータの推定）を行う
1. 学習データのクラスを予測し，いくつ正解したかと正解率（データ数に対する割合）を表示する
1. テストデータについて 2. と同じことをする

#### 問題2

次のセルに，問題1と同様のことを二次判別分析でやるコードを書きなさい．

こちらの場合，`UserWarning: Variables are collinear` という警告が出て，テストデータの予測結果があまり良くないかもしれません．この警告は，変数に多重共線性があるよ，と言っています．

クラス数 $K$，次元数 $D$ のデータに対して，二次判別分析では $D$ 次元の平均を $K$ 個，$D\times D$ の分散共分散行列を $K$ 個推定します．この問題の場合にパラメータの総数を計算すると，

$$
K\times \left(D + \frac{1}{2}D(D+1)\right) = 10 \times (784 + 307720) = 3085040
$$

という膨大な数になります．推定すべきパラメータの数が非常に多く，そのわりにデータ数が少ないため，悪い結果となったと考えられます（このような現象や対処法については，「機械学習I/II」で取り上げます）．

---
### 実験その2

実験その1で出会った問題への対処法の一つは，データの次元を削減して，推定が必要なパラメータ数を減らしてしまう，というものです．主成分分析を適用して次元削減してから判別分析する実験を行ってみましょう．


主成分分析の手続きは自分で書いてもよいですが，ここでは scikit-learn を使って楽をしましょう．
[sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) を使いましょう．リンク先のドキュメントや， notebookA でアヤメのデータの次元削減を行っている部分を参考にしてください．fit でパラメータを推定し，transform で変換するって感じです．

#### 問題3

次のセルに，次のことを行うコードを書きなさい：

1. `X_train` のデータに主成分分析を適用して，累積寄与率が $0.9$ 以上となる最小の次元数 $H$ まで次元を削減する変換を求める．
1. 得られた変換を用いて `X_train` を $H$ 次元に変換したものを変数 `XX_train` に代入する
1. 同様に，`X_test` を変換したものを変数 `XX_test` に代入する
1. `XX_train` と `XX_test` の shape を表示する


上記の 1. のことを実現するには，
```
hoge = PCA(n_components=...)
hoge.fit(X_train)
```
のように `n_components` を指定して [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) の `PCA()` を呼べばok．どのように書くべきかは，リンク先のドキュメントの `Parameters::` の中の `n_components` の説明を読めば分かります．

**注意**: `train` の方で学習（パラメータを推定）するのだから，`test` の方は得られたパラメータを使って変換するだけ．`test` の方で `fit` を呼ぶことはないはず．


#### 問題4

次のセルに，次のことを行うコードを書きなさい
1. 次元削減した学習データを用いて線形判別分析（パラメータの推定）を行う
1. 学習データのクラスを予測し，いくつ正解したかと正解率（データ数に対する割合）を表示する
1. 次元削減したテストデータについて 2. と同じことをする

#### 問題5

次のセルに，問題4と同様のことを二次判別分析でやるコードを書きなさい．


次のセルを実行すると，テストデータの最初の50個と，その

#### 問題6

次のことをやりましょう．

(1) 次のものをメモしておきなさい
- 問題3の次元削減により，何次元のデータが何次元になったか
- 問題1,2,4,5の正解率

(2) 次のセルのコードを修正して実行し，問題5の二次判別の結果が表示されるようにしなさい．その表示を見て，テストデータの最初の50個のうちいくつを間違えたか数え（目視でもコード書くのでもお好きな方でどぞ），その数をメモしておきなさい．

In [ ]:
# テストデータの最初の nx * ny 個を画像として表示
nx, ny = 10, 5
display(X_test[:ny*nx], nx, ny)

# 予測されたクラスラベルを表示
for iy in range(ny):
    print(hoge[iy*nx:(iy+1)*nx])  ### hoge を自分が使った変数名に変えよう

print()
# 正解のクラスラベルを表示
for iy in range(ny):
    print(Y_test[iy*nx:(iy+1)*nx])